## Expermient with RAG offered by Google GenAI (Gemini API)

This notebook demonstrates how to ask questions about a single PDF using Gemini API. \
In this notebook, the PDF is passed in as a file path. PDFs from the web can also be used. \
This code draws inspiration from the [Google Gemini API documentation](https://developers.generativeai.google/api/rest/v1alpha/gemini.projects.locations.models/chat/completions).

Before running this notebook, ensure you have `google-genai` installed. You can install it using pip:

`pip install google-genai`

In [ ]:
from google import genai
from google.genai import types
from pathlib import Path
import os

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
# change this to your file path
file_path = Path(r"../../data/Papers/2412.19437v2 DeepSeek V3 Tech Report 53 pages.pdf")
# Retrieve and encode the PDF byte
file_data = file_path.read_bytes()

In [ ]:
len(file_data)/(1024*1024) # This is the exact size of file in MB

In [ ]:
prompt = "According to the document, how does DeepSeek V3 compare to GPT models in the structure?"

In [12]:
response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[types.Part.from_bytes(data=file_data, mime_type='application/pdf'),prompt])

# this could take anywhere from 15s to 1.5min

In [13]:
from IPython.display import Markdown
Markdown(response.text)

According to the document, DeepSeek-V3 compares to GPT models primarily in its use of specialized architectures for efficiency, rather than a dense Transformer architecture typically associated with models like GPT.

Here are the key structural comparisons:

1.  **Transformer Base:** Both DeepSeek-V3 and GPT models are fundamentally based on the **Transformer framework** (Vaswani et al., 2017).

2.  **Mixture-of-Experts (MoE) Architecture:** This is the most significant structural difference.
    *   **DeepSeek-V3:** It is explicitly designed as a **Mixture-of-Experts (MoE)** language model, using the **DeepSeekMoE** architecture for its Feed-Forward Networks (FFNs). It has 671B total parameters, with only 37B activated per token. DeepSeekMoE uses finer-grained experts and isolates some as shared experts.
    *   **GPT Models:** While the document doesn't explicitly state the architecture of GPT models (e.g., GPT-4, Claude-3.5-Sonnet) beyond performance comparisons, they are generally known to be dense Transformer models, not MoE, unless specified otherwise (e.g., some research versions of GPT might explore MoE). The repeated emphasis on DeepSeek-V3 being an MoE model highlights this as a distinguishing factor.

3.  **Multi-head Latent Attention (MLA):**
    *   **DeepSeek-V3:** It adopts **Multi-head Latent Attention (MLA)**. The core of MLA is a "low-rank joint compression for attention keys and values to reduce Key-Value (KV) cache during inference." This is a specific optimization to the attention mechanism.
    *   **GPT Models:** Typically use standard Multi-Head Attention (MHA), although specific optimizations might exist in closed-source versions.

4.  **Multi-Token Prediction (MTP) Objective:**
    *   **DeepSeek-V3:** Pioneers a **Multi-Token Prediction (MTP)** training objective, which involves additional sequential MTP modules to predict multiple future tokens during training. While these modules are discarded during inference to maintain standard model function, they are part of DeepSeek-V3's *training structure*.
    *   **GPT Models:** Generally trained with a single next-token prediction objective.

5.  **Auxiliary-Loss-Free Load Balancing:**
    *   **DeepSeek-V3:** Introduces an **auxiliary-loss-free strategy** for load balancing its MoE experts, aiming to minimize performance degradation from balancing efforts.
    *   **GPT Models:** As dense models, they don't have the same load balancing challenges inherent to MoE architectures. For other MoE models (not GPT), auxiliary losses are typically used for load balancing.

In summary, DeepSeek-V3 distinguishes itself from typical dense Transformer models like GPT by employing an **MoE architecture for its FFNs**, a specialized **Multi-head Latent Attention**, and a **Multi-Token Prediction training objective**, along with an **auxiliary-loss-free load balancing strategy** to enhance efficiency and performance.